In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('ggplot')


import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir = './Data//paddy-disease-classification/train_images/'
test_dir  = './Data/paddy-disease-classification/test_images/'

BATCH_SIZE = 8
IMG_DIM = 256

In [6]:
img_datagen = ImageDataGenerator(rescale = 1.0 / 255.0,
                                validation_split = .1,
                                rotation_range = .5,
                                shear_range = .3,
                                zoom_range = .3,
                                 width_shift_range = .05,
                                 height_shift_range = .05,
                                 horizontal_flip = True,
                                 vertical_flip = True
                                )

In [7]:
train_gen = img_datagen.flow_from_directory(train_dir,
                                           subset = 'training',
                                           seed = 42,
                                           target_size = (IMG_DIM, IMG_DIM),
                                           batch_size = BATCH_SIZE,
                                           class_mode = 'categorical')

Found 9371 images belonging to 10 classes.


In [8]:
val_gen = img_datagen.flow_from_directory(train_dir,
                                           subset = 'validation',
                                           seed = 42,
                                           target_size = (IMG_DIM, IMG_DIM),
                                           batch_size = BATCH_SIZE,
                                           class_mode = 'categorical')

Found 1036 images belonging to 10 classes.


In [21]:
model = Sequential([
    InputLayer(input_shape = (IMG_DIM, IMG_DIM, 3)),
    Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    BatchNormalization(1),
    MaxPooling2D(3, 3),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    BatchNormalization(1),
    
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    BatchNormalization(1),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    BatchNormalization(1),
    
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    BatchNormalization(1),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    
    #Dense(8126, activation = 'relu'),
    #BatchNormalization(),
    #Dropout(0.5),
    
    Dense(1024, activation = 'relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(10, activation = 'softmax')
])


model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(learning_rate = 1e-3, 
                             decay = 1e-3 / 25),
             metrics = ['acc'])


model.summary()



ResourceExhaustedError: OOM when allocating tensor with shape[56448,8126] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [16]:
EPOCHS = 20

hist = model.fit(train_gen,
                epochs = EPOCHS,
                validation_data = val_gen)

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[56448,8126] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/sequential/dense/MatMul_1 (defined at <ipython-input-16-8be701b04ce1>:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_2383]

Function call stack:
train_function
